In [ ]:
import pandas as pd
import numpy as np
import csv

# ====================================
# Load BTC and ETH close prices
# ====================================
btc = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\BTCUSDT_1m_1000d.csv')
eth = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\ETHUSDT_1m_1000d.csv')

btc_price = btc['close']
eth_price = eth['close']

initial_capital = 1000
trade_size = 200  # maximum notional per trade (USD)


def run_strategy(k: float, fee_rate: float, window_days: int) -> dict:
    """
    Run a simple pair-trading strategy between BTC and ETH.

    The strategy:
    - Compute the spread = BTC - ETH
    - Calculate rolling mean and std over a given window
    - Define upper/lower thresholds = mean ± k * std
    - If spread > upper bound -> sell BTC (partial, $200), buy ETH
    - If spread < lower bound -> sell ETH (partial, $200), buy BTC
    - Each trade pays a fee (fee_rate)

    Parameters
    ----------
    k : float
        Number of standard deviations for threshold.
    fee_rate : float
        Transaction fee rate (e.g., 0.001 = 0.1%).
    window_days : int
        Rolling window size in days (assuming 1-min candles).

    Returns
    -------
    dict : containing performance metrics:
        - k, fee_rate, window_days
        - trades (number of trades executed)
        - final_strat (final portfolio value)
        - return_strat (% return of strategy)
        - return_bh_equal (% return of equal-weight BTC/ETH benchmark)
        - return_bh_btc (% return of buy&hold BTC)
        - return_bh_eth (% return of buy&hold ETH)
        - avg_interval (average minutes between trades)
    """
    # Spread between BTC and ETH
    spread = btc_price - eth_price
    window = 60 * 24 * window_days  # convert days to minutes
    rolling_mean = spread.rolling(window).mean()
    rolling_std = spread.rolling(window).std()

    # Upper / Lower thresholds
    upper_bound = rolling_mean + k * rolling_std
    lower_bound = rolling_mean - k * rolling_std
    zt = spread

    # Initial portfolio: $500 BTC + $500 ETH
    btc_units = 500 / btc_price.iloc[0]
    eth_units = 500 / eth_price.iloc[0]

    portfolio_values = []
    strategy_values = []
    trade_log = []

    for t in range(len(spread)):
        # Benchmark portfolio values
        bh_btc = (500 / btc_price.iloc[0]) * btc_price.iloc[t]
        bh_eth = (500 / eth_price.iloc[0]) * eth_price.iloc[t]
        portfolio_values.append((bh_btc, bh_eth))

        total_value = btc_units * btc_price.iloc[t] + eth_units * eth_price.iloc[t]

        if not np.isnan(upper_bound.iloc[t]):
            if zt.iloc[t] > upper_bound.iloc[t]:
                # BTC expensive -> sell BTC, buy ETH
                btc_value = btc_units * btc_price.iloc[t]
                if btc_value > 0:
                    sell_value = min(trade_size, btc_value)
                    fee = sell_value * fee_rate
                    sell_value_after_fee = sell_value - fee
                    btc_units -= sell_value / btc_price.iloc[t]
                    eth_units += sell_value_after_fee / eth_price.iloc[t]
                    trade_log.append(t)

            elif zt.iloc[t] < lower_bound.iloc[t]:
                # BTC cheap -> sell ETH, buy BTC
                eth_value = eth_units * eth_price.iloc[t]
                if eth_value > 0:
                    sell_value = min(trade_size, eth_value)
                    fee = sell_value * fee_rate
                    sell_value_after_fee = sell_value - fee
                    eth_units -= sell_value / eth_price.iloc[t]
                    btc_units += sell_value_after_fee / btc_price.iloc[t]
                    trade_log.append(t)

        strat_value = btc_units * btc_price.iloc[t] + eth_units * eth_price.iloc[t]
        strategy_values.append(strat_value)

    # Benchmark final values
    bh_btc_value = [x[0] for x in portfolio_values]
    bh_eth_value = [x[1] for x in portfolio_values]
    bh_equal = [x[0] + x[1] for x in portfolio_values]

    final_btc = bh_btc_value[-1]
    final_eth = bh_eth_value[-1]
    final_equal = bh_equal[-1]
    final_strat = strategy_values[-1]

    # Trade interval stats
    if len(trade_log) > 1:
        trade_intervals = np.diff(trade_log)
        avg_interval = np.mean(trade_intervals)
    else:
        avg_interval = None

    return {
        "k": k,
        "fee_rate": fee_rate,
        "window_days": window_days,
        "trades": len(trade_log),
        "final_strat": final_strat,
        "return_strat": (final_strat / initial_capital - 1) * 100,
        "return_bh_equal": (final_equal / initial_capital - 1) * 100,
        "return_bh_btc": (final_btc / 500 - 1) * 100,
        "return_bh_eth": (final_eth / 500 - 1) * 100,
        "avg_interval": avg_interval,
    }


# ====================================
# 3. Grid Search
# ====================================
k_list = [0.5, 1, 1.5, 2]
fee_list = [0.0005, 0.001, 0.002]  # 0.05% to 0.2%
window_list = [5, 10, 20]          # 5, 10, 20-day windows

output_file = "grid_results.csv"

with open(output_file, "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=[
            "k",
            "fee_rate",
            "window_days",
            "trades",
            "final_strat",
            "return_strat",
            "return_bh_equal",
            "return_bh_btc",
            "return_bh_eth",
            "avg_interval",
        ],
    )
    writer.writeheader()

    # Iterate over all parameter combinations
    for k in k_list:
        for fee in fee_list:
            for win in window_list:
                result = run_strategy(k, fee, win)
                writer.writerow(result)
                print("Done:", result)
